In [1]:
%run ../commons/imports.ipynb
%run ../commons/helper_db.ipynb
%run ../commons/helper_contents.ipynb

In [2]:
MAIN_FOLDER = main_folder("cannabisroad2")

create_database('blp_dataset')
create_table('products_tmp_cannabisroad')

In [7]:
folders = [x for x in Path(MAIN_FOLDER).iterdir() if x.is_dir()]

for fo in folders:
    folder = Path(f"{fo}")
    folder_image = Path(f"{fo}")

    if folder.exists():
        for file in os.listdir(folder):
            page_content = BeautifulSoup(open(Path(f"{folder}\{file}"), encoding='utf8').read())

            if page_content is not None:
                ## GET DESCRIPTION
                description = ""
                raw_desc = page_content.find("pre")

                if raw_desc is not None:
                    description = get_content(raw_desc)

                ## GET TITLE
                title = ""
                raw_title = page_content.find("title")
                
                if raw_title is not None:
                    title = get_content(raw_title)
                    
                ## GET SELLER
                seller = ""
                raw_seller = page_content.find("a", attrs={"class": "toplink"})
                
                if raw_seller is not None:
                    seller = get_content(raw_seller)
                    
                ## GET PRICE
                price = ""
                raw_price = page_content.find("span", attrs={"class": "price"})
                
                if raw_price is not None:
                    price = get_content(raw_price)

                if description != "" or title != "":
                    # Insert product into database
                    query = "INSERT INTO products_tmp_cannabisroad (name, description, market_name, price, seller_name, illegal) VALUES (%s, %s, %s, %s, %s, %s)"
                    values = (title, description, "Cannabis Road 2", price, seller, "t")

                    cursor.execute(query, values)
                    conn.commit()

                    product_id = cursor.lastrowid

                    ## GET IMAGE
                    raw_image = page_content.find("div", attrs={"class": "product-img-box"})
                    
                    if raw_image is not None:
                        img_content = raw_image.find("img")

                        if img_content is not None:
                            image = img_content["src"]

                            try:
                                data = "" + image.replace("data:image/jpeg;base64,", "")
                                imgdata = base64.b64decode(data)

                                img_filename = Path(f"D:\images\_cannabis_road_2\{product_id}.jpeg")
                                
                                print(f"{product_id}")
                                print(f"{folder_image}\{file}")

                                with open(img_filename, 'wb') as f:
                                    f.write(imgdata)
                                    f.close
                            except:
                                print(f"Image {product_id} has error. Except")
                                print(f"{folder_image}\{file}")
                                update_image(product_id, "no_image")
                        else:
                            print(f"Product {product_id} has no image")
                            update_image(product_id, "no_image")
                    else:
                        print(f"Product {product_id} has no image")
                        update_image(product_id, "no_image") 

1
D:\dnmarchives\dnmarchives\cannabisroad2\shop\100g-landrace-hash-philippines
2
D:\dnmarchives\dnmarchives\cannabisroad2\shop\100g-pollen-hash-philippines
Image 3 has error. Except
D:\dnmarchives\dnmarchives\cannabisroad2\shop\100thctoffee
4
D:\dnmarchives\dnmarchives\cannabisroad2\shop\10DecaDoseCheebaChew
Image 5 has error. Except
D:\dnmarchives\dnmarchives\cannabisroad2\shop\10g-Indica-Hash-HQ
6
D:\dnmarchives\dnmarchives\cannabisroad2\shop\10g-landrace-hash-philippines
7
D:\dnmarchives\dnmarchives\cannabisroad2\shop\10g-pollen-hash-philippines
Image 8 has error. Except
D:\dnmarchives\dnmarchives\cannabisroad2\shop\10g-Sativa-Hash-HQ
Image 9 has error. Except
D:\dnmarchives\dnmarchives\cannabisroad2\shop\10gsobfs
Image 10 has error. Except
D:\dnmarchives\dnmarchives\cannabisroad2\shop\10honeycomb
11
D:\dnmarchives\dnmarchives\cannabisroad2\shop\10Incredibles100mgBars
Image 12 has error. Except
D:\dnmarchives\dnmarchives\cannabisroad2\shop\10joints
13
D:\dnmarchives\dnmarchives\cann

107
D:\dnmarchives\dnmarchives\cannabisroad2\shop\2GSquidyBlacjk
Image 108 has error. Except
D:\dnmarchives\dnmarchives\cannabisroad2\shop\2ozescrow
Image 109 has error. Except
D:\dnmarchives\dnmarchives\cannabisroad2\shop\2ozfe
Image 110 has error. Except
D:\dnmarchives\dnmarchives\cannabisroad2\shop\2ozhash
Image 111 has error. Except
D:\dnmarchives\dnmarchives\cannabisroad2\shop\2ozkief
112
D:\dnmarchives\dnmarchives\cannabisroad2\shop\2ozwax
Image 113 has error. Except
D:\dnmarchives\dnmarchives\cannabisroad2\shop\2Shatter
Image 114 has error. Except
D:\dnmarchives\dnmarchives\cannabisroad2\shop\30g-Indica-Hash-HQ
Image 115 has error. Except
D:\dnmarchives\dnmarchives\cannabisroad2\shop\30g-Sativa-Hash-HQ
116
D:\dnmarchives\dnmarchives\cannabisroad2\shop\35bho
Image 117 has error. Except
D:\dnmarchives\dnmarchives\cannabisroad2\shop\35blue
Image 118 has error. Except
D:\dnmarchives\dnmarchives\cannabisroad2\shop\35BW
Image 119 has error. Except
D:\dnmarchives\dnmarchives\cannabisro

Image 211 has error. Except
D:\dnmarchives\dnmarchives\cannabisroad2\shop\AK47cross
Image 212 has error. Except
D:\dnmarchives\dnmarchives\cannabisroad2\shop\AK47HP
Image 213 has error. Except
D:\dnmarchives\dnmarchives\cannabisroad2\shop\AK47QP
Image 214 has error. Except
D:\dnmarchives\dnmarchives\cannabisroad2\shop\Alienogshatter
Image 215 has error. Except
D:\dnmarchives\dnmarchives\cannabisroad2\shop\alienogshatter2
Image 216 has error. Except
D:\dnmarchives\dnmarchives\cannabisroad2\shop\AlienShatter
Image 217 has error. Except
D:\dnmarchives\dnmarchives\cannabisroad2\shop\AlienShatter2
Image 218 has error. Except
D:\dnmarchives\dnmarchives\cannabisroad2\shop\allinone-cheap-weed-pricey-mix
Image 219 has error. Except
D:\dnmarchives\dnmarchives\cannabisroad2\shop\amberwax
Image 220 has error. Except
D:\dnmarchives\dnmarchives\cannabisroad2\shop\assortedseeds
221
D:\dnmarchives\dnmarchives\cannabisroad2\shop\autoNLxLR
Image 222 has error. Except
D:\dnmarchives\dnmarchives\cannabisr

Image 310 has error. Except
D:\dnmarchives\dnmarchives\cannabisroad2\shop\Charas14g
Image 311 has error. Except
D:\dnmarchives\dnmarchives\cannabisroad2\shop\Charas1g
Image 312 has error. Except
D:\dnmarchives\dnmarchives\cannabisroad2\shop\charas1gram
Image 313 has error. Except
D:\dnmarchives\dnmarchives\cannabisroad2\shop\charas25gram
Image 314 has error. Except
D:\dnmarchives\dnmarchives\cannabisroad2\shop\Charas28g
Image 315 has error. Except
D:\dnmarchives\dnmarchives\cannabisroad2\shop\charas3g
Image 316 has error. Except
D:\dnmarchives\dnmarchives\cannabisroad2\shop\charas50gram
Image 317 has error. Except
D:\dnmarchives\dnmarchives\cannabisroad2\shop\charas7g
Image 318 has error. Except
D:\dnmarchives\dnmarchives\cannabisroad2\shop\Charras5gram
Image 319 has error. Except
D:\dnmarchives\dnmarchives\cannabisroad2\shop\Cheap
Image 320 has error. Except
D:\dnmarchives\dnmarchives\cannabisroad2\shop\CheebaChewCaramelIndicaExtraSt
Image 321 has error. Except
D:\dnmarchives\dnmarchi

412
D:\dnmarchives\dnmarchives\cannabisroad2\shop\emc05g
413
D:\dnmarchives\dnmarchives\cannabisroad2\shop\emc27g
414
D:\dnmarchives\dnmarchives\cannabisroad2\shop\emc28g
415
D:\dnmarchives\dnmarchives\cannabisroad2\shop\emc2g
416
D:\dnmarchives\dnmarchives\cannabisroad2\shop\Five250mgOpenCartridges
417
D:\dnmarchives\dnmarchives\cannabisroad2\shop\Five500mgOpenCartridges
Image 418 has error. Except
D:\dnmarchives\dnmarchives\cannabisroad2\shop\fivejointsescrow
Image 419 has error. Except
D:\dnmarchives\dnmarchives\cannabisroad2\shop\fivejointsfe
Image 420 has error. Except
D:\dnmarchives\dnmarchives\cannabisroad2\shop\FreeSample
Image 421 has error. Except
D:\dnmarchives\dnmarchives\cannabisroad2\shop\freesampleafghan
Image 422 has error. Except
D:\dnmarchives\dnmarchives\cannabisroad2\shop\freesampleedrosenthal
423
D:\dnmarchives\dnmarchives\cannabisroad2\shop\Freesamplemarochash
Image 424 has error. Except
D:\dnmarchives\dnmarchives\cannabisroad2\shop\freesamplemazargr0wer
Image 425

Image 506 has error. Except
D:\dnmarchives\dnmarchives\cannabisroad2\shop\JackHerer
Image 507 has error. Except
D:\dnmarchives\dnmarchives\cannabisroad2\shop\JackHerrer
Image 508 has error. Except
D:\dnmarchives\dnmarchives\cannabisroad2\shop\JackTheRipper
Product 509 has no image
Image 510 has error. Except
D:\dnmarchives\dnmarchives\cannabisroad2\shop\joint
Image 511 has error. Except
D:\dnmarchives\dnmarchives\cannabisroad2\shop\joints
512
D:\dnmarchives\dnmarchives\cannabisroad2\shop\jointWeedCannabis
513
D:\dnmarchives\dnmarchives\cannabisroad2\shop\jointweedcannabis1
514
D:\dnmarchives\dnmarchives\cannabisroad2\shop\jointweedcannabis2
Image 515 has error. Except
D:\dnmarchives\dnmarchives\cannabisroad2\shop\JTR
Image 516 has error. Except
D:\dnmarchives\dnmarchives\cannabisroad2\shop\JTRipper
517
D:\dnmarchives\dnmarchives\cannabisroad2\shop\ketama-10
518
D:\dnmarchives\dnmarchives\cannabisroad2\shop\ketama-100
519
D:\dnmarchives\dnmarchives\cannabisroad2\shop\ketama-25
520
D:\dn

Image 615 has error. Except
D:\dnmarchives\dnmarchives\cannabisroad2\shop\Nepalise
616
D:\dnmarchives\dnmarchives\cannabisroad2\shop\NSTE123
617
D:\dnmarchives\dnmarchives\cannabisroad2\shop\NSTER123
618
D:\dnmarchives\dnmarchives\cannabisroad2\shop\NSTER1234
619
D:\dnmarchives\dnmarchives\cannabisroad2\shop\OgKushPenHashOil
Image 620 has error. Except
D:\dnmarchives\dnmarchives\cannabisroad2\shop\ogsharksample
621
D:\dnmarchives\dnmarchives\cannabisroad2\shop\oldschoolrocky10g
Image 622 has error. Except
D:\dnmarchives\dnmarchives\cannabisroad2\shop\onegrambhowax
623
D:\dnmarchives\dnmarchives\cannabisroad2\shop\OneOpenOneCartridge250mg
Image 624 has error. Except
D:\dnmarchives\dnmarchives\cannabisroad2\shop\OneOunceAfghani
Image 625 has error. Except
D:\dnmarchives\dnmarchives\cannabisroad2\shop\OneOunceLemonKush
Image 626 has error. Except
D:\dnmarchives\dnmarchives\cannabisroad2\shop\oneozkushmes
Image 627 has error. Except
D:\dnmarchives\dnmarchives\cannabisroad2\shop\oneozsativa

713
D:\dnmarchives\dnmarchives\cannabisroad2\shop\sampleoflemonogwax
714
D:\dnmarchives\dnmarchives\cannabisroad2\shop\sampleofwhiterussian
Image 715 has error. Except
D:\dnmarchives\dnmarchives\cannabisroad2\shop\sandscandy
Image 716 has error. Except
D:\dnmarchives\dnmarchives\cannabisroad2\shop\sativa
Image 717 has error. Except
D:\dnmarchives\dnmarchives\cannabisroad2\shop\seed100
Image 718 has error. Except
D:\dnmarchives\dnmarchives\cannabisroad2\shop\seed20
Image 719 has error. Except
D:\dnmarchives\dnmarchives\cannabisroad2\shop\seed50
Image 720 has error. Except
D:\dnmarchives\dnmarchives\cannabisroad2\shop\seededsilverhaze
Image 721 has error. Except
D:\dnmarchives\dnmarchives\cannabisroad2\shop\seeds10
Image 722 has error. Except
D:\dnmarchives\dnmarchives\cannabisroad2\shop\seedseedsoutdoorsativabigyield
Image 723 has error. Except
D:\dnmarchives\dnmarchives\cannabisroad2\shop\SFVog
Image 724 has error. Except
D:\dnmarchives\dnmarchives\cannabisroad2\shop\shake
Image 725 ha

814
D:\dnmarchives\dnmarchives\cannabisroad2\shop\water215
815
D:\dnmarchives\dnmarchives\cannabisroad2\shop\water3
816
D:\dnmarchives\dnmarchives\cannabisroad2\shop\water4
817
D:\dnmarchives\dnmarchives\cannabisroad2\shop\water5
818
D:\dnmarchives\dnmarchives\cannabisroad2\shop\water7
819
D:\dnmarchives\dnmarchives\cannabisroad2\shop\water8
820
D:\dnmarchives\dnmarchives\cannabisroad2\shop\water98
821
D:\dnmarchives\dnmarchives\cannabisroad2\shop\water9888
Image 822 has error. Except
D:\dnmarchives\dnmarchives\cannabisroad2\shop\wax
823
D:\dnmarchives\dnmarchives\cannabisroad2\shop\WaxBho
824
D:\dnmarchives\dnmarchives\cannabisroad2\shop\waxcrumbleshatterthccannabisco
825
D:\dnmarchives\dnmarchives\cannabisroad2\shop\waxcrumblethccannabistopblonde
Image 826 has error. Except
D:\dnmarchives\dnmarchives\cannabisroad2\shop\WaxDabs
827
D:\dnmarchives\dnmarchives\cannabisroad2\shop\waxlottery1oz
828
D:\dnmarchives\dnmarchives\cannabisroad2\shop\waxlotteryhalfoz
829
D:\dnmarchives\dnmarchiv